In [1]:
%load_ext autoreload

In [10]:
%autoreload 2

import h5py
import hydra
from JacobianODE.jacobians.jacobian_utils import initialize_config, make_trajectories, postprocess_data, normalize_data, create_dataloaders, setup_wandb, make_model, log_training_info, train_model
import numpy as np
from omegaconf import OmegaConf
import os
import torch

In [3]:
# FILL THIS IN FOR YOUR OWN USE
save_dir = "/orcd/data/ekmiller/001/eisenaj/JacobianODE/lightning"

In [11]:
session_dir = "/orcd/data/ekmiller/001/common/datasets/anesthesia/mat/propofolPuffTone"
session_file = "Mary-Anesthesia-20160912-02.mat"

In [13]:
# open the session file

session_path = os.path.join(session_dir, session_file)
f = h5py.File(session_path, "r")

# print the keys
print(f.keys())

<KeysViewHDF5 ['#refs#', '#subsystem#', 'ain', 'ainSchema', 'amua', 'amuaSchema', 'analogChnlInfo', 'electrodeInfo', 'eventSchema', 'fileInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeChnlInfo', 'spikeTimes', 'spikeTimesSchema', 'spikeWaves', 'spikeWavesSchema', 'trialInfo', 'unitInfo']>


In [24]:
eyes_open = f['sessionInfo']['eyesClose'][-1, -1]
eyes_close = f['sessionInfo']['eyesClose'][-1, -1]

In [28]:
length = 60 # seconds
subsample = 1 # time steps

In [38]:
awake_start_time = int(eyes_close - 15*60)
dt = f['lfpSchema']['smpInterval'][0, 0]

In [41]:
start_ind, end_ind = int(awake_start_time/dt), int((awake_start_time + length)/dt)

In [47]:
lfp_data = f['lfp'][:, start_ind:end_ind].T
lfp_data = lfp_data[::subsample, :]

In [48]:
lfp_data.shape

(60000, 249)

## Setup

In [53]:
# load the config
with hydra.initialize(version_base="1.3", config_path="../JacobianODE/jacobians/conf/"):
    cfg = hydra.compose(config_name="config", overrides=["data=custom"])
# print the config in a nice format


# Print the config in a nice format
print(OmegaConf.to_yaml(cfg))

logger: wandb
training:
  batch_size: 16
  run_number: 0
  lightning:
    _target_: null
    direct: true
    loss_func: mse
    alpha_hal: 0.1
    l2_penalty: 0
    l1_penalty: 0.0
    obs_noise_scale: 0
    final_obs_noise_scale: 0
    y0_noise_scale: 0
    noise_annealing: false
    log_interval: 100
    alpha_teacher_forcing: 1
    teacher_forcing_annealing: true
    gamma_teacher_forcing: 0.999
    teacher_forcing_update_interval: 5
    teacher_forcing_steps: 1
    min_alpha_teacher_forcing: 0
    alpha_validation: 0
    obs_noise_scale_validation: 0
    loss_func_validation: mse
    traj_init_steps_validation: 15
    inner_N_validation: 20
    data_type: null
    jacobianODEint_kwargs:
      traj_init_steps: 15
      inner_path: line
      inner_N: 20
      interp_pts: 4
    gradient_clip_val: 1.0
    gradient_clip_algorithm: norm
    optimizer: AdamW
    optimizer_kwargs:
      lr: 0.0001
      weight_decay: 0.0001
    use_scheduler: true
    min_lr: 1.0e-06
    k_scale: 1
    j

In [49]:
cfg.training.logger.save_dir = save_dir

In [50]:
# initial setup
torch.set_float32_matmul_precision('high')
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

Number of available GPUs: 1


In [ ]:
# initialize the config
cfg = initialize_config(cfg)